In [1]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///tmp/build/80754af9/scikit-learn_1642617106979/work
scikit-learn-intelex==2021.20220215.212715


In [2]:
import pickle
import pandas as pd

In [3]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [4]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [5]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [6]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [7]:
# Q1
y_pred.mean()

16.191691679979066

In [15]:
# Q2
year = 2021
month = 2
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [18]:
df_result = df[['ride_id']]
df_result['pred'] = y_pred

/tmp/ipykernel_2659/3048028083.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_result['pred'] = y_pred


In [21]:
output_file = './predictions_202102.csv'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

# 18.8 MB

##### Q3

`jupyter nbconvert --to=script starter.ipynb`

##### Q4
`pipenv install scikit-learn==1.0.2 pandas --python=3.9`
> sha256:08ef968f6b72033c16c479c966bf37ccd49b06ea91b765e1cc27afefe723920b

##### Q5
> 16.30

##### Q6
`docker build -t ride-duration-prediction:v1 .`
`docker run ride-duration-prediction:v1`
> 9.97